In [21]:
import pandas as pd
import numpy as np
import os

from MOA_L1000 import validation, config
import MOA_L1000.dataprep as dataprep
import MOA_L1000.dataload as dataload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:

def load_prediction(config_id=1):
    """Load params for the run, load ensemble solution and holdout metrics"""
    fcn_config = eval("config.get_params_{}".format(config_id))
    params = fcn_config()
    directory = params.get("output_directory")
    dict_solutions = {'df_ensemble':None, 'df_ensemble_mean':None, 'df_ensemble_median':None}
    
    filename = os.path.join(directory, "df_solution_ensemble.h5")
    if os.path.exists(filename):
        df_ensemble = pd.read_hdf(filename, 'root').astype('float64')
        dict_solutions['df_ensemble'] = df_ensemble
    filename = os.path.join(directory, "df_solution_ensemble_mean.h5")
    if os.path.exists(filename):
        df_ensemble = pd.read_hdf(filename, 'root').astype('float64')
        dict_solutions['df_ensemble_mean'] = df_ensemble
    filename = os.path.join(directory, "df_solution_ensemble_median.h5")
    if os.path.exists(filename):
        df_ensemble = pd.read_hdf(filename, 'root').astype('float64')
        dict_solutions['df_ensemble_median'] = df_ensemble
        
    filename = os.path.join(directory, "holdout_metrics.csv")
    df_metrics = pd.read_csv(filename, sep="\t", index_col=0)
    return dict_solutions, df_metrics, params


def compute_precision_and_recall(df_solution, df_ground=None):
    """Load solution matrix. Load params for our model. Load
    ensemble solution for our model. Load holdout metrics for our model. """
    assert df_ground is not None
    df_g = df_ground
    df_s = df_solution.loc[df_g.index, :]
    precisions = validation.compute_precision(df_s, df_g)
    recalls = validation.compute_recall(df_s, df_g)
    return precisions, recalls


def load_all_predictions_and_metrics(config_ids = []):
    """ For all solutions, load the ensemble solution, 
     and the holdout metrics"""
    solutions = {}
    metrics = {}
    for config_id in config_ids:
        dict_solutions, df_metrics, params = load_prediction(config_id)
        solutions[config_id] = dict_solutions
        metrics[config_id] = df_metrics
    
    df_metrics = pd.DataFrame(
        {config_id:d['holdout_score'] for config_id, d in metrics.items()}
        )
    return solutions, df_metrics

In [76]:
config_ids = ['DNN1', "DNN2", "DNN3", "DNN4", "DNN5", "DNN6", "DNN7", "DNN8", "DNN9", "DNN10", "DNN11", "DNN12", "DNN13", "DNN14", "DNN15", "DNN17"]
predictions, df_metrics = load_all_predictions_and_metrics(config_ids)
df_metrics

,DNN1,DNN2,DNN3,DNN4,DNN5,DNN6,DNN7,DNN8,DNN9,DNN10,DNN11,DNN12,DNN13,DNN14,DNN15,DNN17
0_0,0.05419,0.05138,0.04700,0.04875,0.05168,0.04791,0.04737,0.04737,0.05253,0.05565,0.05407,0.05704,0.05017,0.05030,0.04725,0.04559
0_1,0.05362,0.05431,0.04883,0.04779,0.05188,0.04760,0.04857,0.04693,0.05220,0.05718,0.05807,0.05932,0.05277,0.05157,0.04671,0.04629
0_2,0.05524,0.05703,0.04719,0.04905,0.05018,0.04853,0.04735,0.04707,0.05496,0.05593,0.05611,0.05926,0.05373,0.05347,0.04588,0.04624
0_3,0.05382,0.05237,0.04845,0.04695,0.05395,0.04773,0.04839,0.04721,0.05440,0.05552,0.05444,0.05828,0.05228,0.05075,0.04538,0.04481
0_4,0.05772,0.05282,0.04834,0.04952,0.05258,0.04770,0.04821,0.04798,0.05342,0.05659,0.05691,0.05918,0.05280,0.05210,0.04756,0.05167
1_0,0.05342,0.05590,0.04570,0.04850,0.05531,0.04816,0.04835,0.04710,0.05415,0.05612,0.05799,0.05654,0.05126,0.05152,0.04486,0.04696
1_1,0.05651,0.05424,0.04562,0.04916,0.05109,0.04759,0.04805,0.04590,0.05178,0.05755,0.05786,0.05939,0.05214,0.05316,0.04541,0.04672
1_2,0.05306,0.05203,0.04630,0.04851,0.05099,0.04810,0.04845,0.04749,0.05402,0.05604,0.05372,0.06003,0.05310,0.05182,0.04609,0.04643
1_3,0.05940,0.05131,0.04625,0.04772,0.05280,0.04751,0.04935,0.04808,0.05581,0.05571,0.05588,0.05792,0.05282,0.05031,0.04621,0.04617
1_4,0.04976,0.05479,0.04901,0.04934,0.05033,0.04847,0.04681,0.04691,0.05356,0.05936,0.05044,0.05782,0.05058,0.05110,0.04567,0.04618


In [78]:
X_train, X_holdout, w_train, y_train, y_holdout = dataload.load_data_v1()
dict_solutions, df_metrics, params = load_prediction('DNN17')
df_preds = dict_solutions['df_ensemble_median']
precisions, recalls = compute_precision_and_recall(df_preds, y_holdout)
metrics = pd.merge(precisions, recalls, on="moa")


In [82]:
metrics.tail(30).sort_values('precision')
metrics.tail(30).sort_values('recall')

,precision,recall
moa,,
ACETYLCHOLINE RECEPTOR AGONIST,0.000000,0.000000
CHELATING AGENT,0.000000,0.000000
HMGCR INHIBITOR,0.000000,0.000000
ANDROGEN RECEPTOR ANTAGONIST,0.250000,0.111111
DNA INHIBITOR,0.272727,0.142857
DOPAMINE RECEPTOR ANTAGONIST,0.038835,0.166667
KIT INHIBITOR,0.333333,0.166667
SRC INHIBITOR,1.000000,0.166667
CYCLOOXYGENASE INHIBITOR,0.064516,0.222222


In [45]:
df_preds

CCK RECEPTOR ANTAGONIST  PLK INHIBITOR  \
pert_id       pert_idose                                           
BRD-A02006392 1.11 uM                    0.001596       0.000692   
              10 uM                      0.003545       0.000993   
              3.33 uM                    0.000999       0.000293   
BRD-A03216249 1.11 uM                    0.139555       0.003569   
              10 uM                      0.012110       0.003135   
...                                           ...            ...   
BRD-K99792991 10 uM                      0.000091       0.001190   
              3.33 uM                    0.000990       0.003885   
BRD-K99964838 1.11 uM                    0.001509       0.002688   
              10 uM                      0.000374       0.003213   
              3.33 uM                    0.003087       0.007740   

                          TRICYCLIC ANTIDEPRESSANT  CHOLESTEROL INHIBITOR  \
pert_id       pert_idose                                                    
BRD-A02006392 1.11 uM                     0.001914               0.001720   
              10 uM                       0.003025               0.001784   
              3.33 uM                     0.002145               0.000929   
BRD-A03216249 1.11 uM                     0.003035               0.003739   
              10 uM                       0.006917               0.007079   
...                                            ...                    ...   
BRD-K99792991 10 uM                       0.000118               0.000009   
              3.33 uM                     0.002636               0.000280   
BRD-K99964838 1.11 uM                     0.007831               0.002437   
              10 uM                       0.004124               0.000082   
              3.33 uM                     0.015034               0.002653   

                          ATP CHANNEL BLOCKER  \
pert_id       pert_idose                        
BRD-A02006392 1.11 uM                0.001825   
              10 uM                  0.004690   
              3.33 uM                0.001314   
BRD-A03216249 1.11 uM                0.002245   
              10 uM                  0.007220   
...                                       ...   
BRD-K99792991 10 uM                  0.000054   
              3.33 uM                0.000478   
BRD-K99964838 1.11 uM                0.002441   
              10 uM                  0.000355   
              3.33 uM                0.003964   

                          THROMBOXANE RECEPTOR ANTAGONIST  SRC INHIBITOR  \
pert_id       pert_idose                                                   
BRD-A02006392 1.11 uM                            0.004749       0.002169   
              10 uM                              0.004297       0.001726   
              3.33 uM                            0.004420       0.000632   
BRD-A03216249 1.11 uM                            0.004004       0.001535   
              10 uM                              0.014371       0.001967   
...                                                   ...            ...   
BRD-K99792991 10 uM                              0.000035       0.000363   
              3.33 uM                            0.001189       0.003644   
BRD-K99964838 1.11 uM                            0.003377       0.003778   
              10 uM                              0.000174       0.016739   
              3.33 uM                            0.004122       0.015413   

                          AROMATASE INHIBITOR  OPIOID RECEPTOR ANTAGONIST  \
pert_id       pert_idose                                                    
BRD-A02006392 1.11 uM                0.002727                    0.002826   
              10 uM                  0.002954                    0.002501   
              3.33 uM                0.001586                    0.002727   
BRD-A03216249 1.11 uM                0.001603                    0.001525   
              10 uM                  0.004164     